# [オンライン開催]自然言語処理ライブラリTransformersを学ぶ第1回
カジュアルに、勉強会を開催していきます。当日はオンライン開催となりますが、質疑応答については別途、勉強会が終わり次第、オープンコミュニケーションツールSlackにて対応いたします（直接勉強会に参加された方のみ、招待リンクを送付しています）。また、勉強会にあたって、皆様のプライバシーを保護する観点から、ボイスとカメラはOFFとなります。

## Section0. 概要

機械翻訳タスクにおいて、翻訳品質と学習コストの両面でリカレントニューラルネット（RNN）を上回る、Transformer と名付けられたニューラルネットアーキテクチャが2017年Googleから論文発表されました。

これと並行して、ULMFiTと呼ばれる効果的な転移学習法により、非常に大規模かつ多様なコーパスでLSTM(Long Short-Term Memory: 長短期記憶)ネットワークを学習すれば、少ないラベル付きデータで性能の良いテキスト分類器を構築できることが示されました。

これらの研究は、今日もっともよく知られている２つの Transformer である、Generative Pretrained Transformer (GPT) と、Bi-directional Encoder Representations from Transformers (BERT) のきっかけとなりました。Transformer と教師なし学習と組み合わせることで、これらのモデルはタスク固有のアーキテクチャをゼロから学習する必要性をなくし、NLPのほぼ全てのベンチマークを大幅に更新しました。

GPTと、BERTのリリース以来、Transformerを元としたモデルが多く提案されています。

![任意の画像名を付ける](https://drive.google.com/uc?id=1R13KLqKl6WQnjoNP6znQ5a3mZv-sRmBB)
【出典: 機械学習エンジニアのためのTransformers 図1-1. Transformer の年表】

## Section1. エンコーダ・デコーダのフレームワーク

Transformer 以前は、LSTMのようなリカレントニューラルネットが自然言語処理における最先端な技術でした。これらのアーキテクチャはネットワーク接続にフィードバックループをもち、ある時刻から別の時刻へと情報を伝搬させ、かつ比較的長期に情報を"記憶"させることができるものです。したがって、テキストのような連続したデータのモデル化に理想的なものです。  

下図の左側に示す通り、リカレントニューラルネットはある入力（単語や文字）を受け取り、それをネットワークを通して、**隠れ状態**(hidden state)と呼ばれるベクトルを出力します。同時に、このモデルはフィードバックループを通して、自分自身に情報をフィードバックし、次の時刻でその情報を利用できます。

下図の右側に示すように、ループを「展開」すると、RNNは各時刻の状態に関する情報を次の演算に渡していることが、より明確にわかります。これにより、リカレントニューラルネットは前の時刻の情報を記録しておき、予測に利用できます。

![任意の画像名を付ける](https://drive.google.com/uc?id=14rAJEWHIV3uNz-SP67x00jJB0jcVF-79)  

リカレントニューラルネットは、自然言語処理、音声処理、時系列処理などに広く利用されてきました。  
(参考: [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/))  

リカレントニューラルネットが重要な役割を果たした分野の１つに、ある言語の単語列を別の言語に対応づけることを目的とした機械翻訳システムの開発があります。この種のタスクは通常、エンコード・デコード(encoder-decoder)または Seq2seq(sequence-to-sequence) アーキテクチャで取り組まれ、入力と出力が両方とも任意の長さの系列である場合に適しています。

エンコーダの仕事は、入力系列から与えられる情報を、**最後の隠れ状態**(last hidden state) と呼ばれる数値表現にエンコードすることです。この状態はデコードに渡され、デコーダは出力系列を生成します。  

一般に、エンコーダとデコーダの構成要素は、系列をモデル化できるのであれば、どんなニューラルネットワークのアーキテクチャでも使えます。系列のモデル化は、下図のようになります。下の例では、"Transformers are great!" が隠れ状態ベクトルとしてエンコードされ、それがデコードされてドイツ語の翻訳「Transformer sind grossartig!」が生成されます。入力語はエンコーダに順次与えられ、出力語は上から下へ１つずつ生成されます。

![任意の画像名を付ける](https://drive.google.com/uc?id=1kXYntNMoP9MuwR0Pyy-yr9tbwNx4QW48)

このアーキテクチャのデメリットは、最後の隠れ状態において、デコーダが出力する時に利用されるもので、入力全体の意味を表現する必要があります。しかし、長い系列データになればなるほど、意味を表現することが困難となり、全ての情報を１つの表現に圧縮する過程で、入力開始時点での情報を失うことがあります。

この欠点を解消すべく、**アテンション機構（注意機構）**なるものがあります。






## Section2. アテンション機構

アテンションの主な考え方は、入力系列データに対して単一の隠れ状態を生成する代わりに、エンコーダはデコーダがアクセスできる隠れ状態を各時刻で出力するというものになります。

しかしながら、全ての状態を同時に使用すると、デコーダの入力が膨大となるために、どの状態を優先すべきか（注意すべきか）といった順位づけが必要となります。そこで、デコーダとエンコーダの核状態に、デコードの時刻ごとに異なる重み、つまり『アテンション』を割り当てることにします。（下図）
  
![任意の画像名を付ける](https://drive.google.com/uc?id=1dQMCnTJjXnbSwgavNChVKTqDhYVQr7jm)

各時刻で、どの入力トークンの関連性が最も高いかについて注目すること（注意すること）で、アテンションに基づくモデルは、生成された翻訳文の単語と原文の単語の間のアライメントを学習できます。

例えば、下図は英仏翻訳モデルにおけるアテンションの重み（各ピクセル）を可視化したものです。この図は、２つの言語で順序が異なる "zone", "Area" という単語のアライメントを、デコーダが正しく表現できていることを示しています。

![任意の画像名を付ける](https://drive.google.com/uc?id=1oUtiXmkbsIJTvuA_GAYs8NylRHIlPWYS)

アテンションを用いることで、優れた翻訳を生成できる一方、エンコーダとデコーダにリカレントニューラルネットを使用した場合には大きな欠点がありました。それは、これらのモデルでは逐次的に計算するために、入力系列データ全体で並列化ができないという点です。

Transformer によって、新しい変革を促されました。それは、再帰性を完全に排除して、代わりに**セルフアテンション**（self-attention） と呼ばれます。

基本的な考え方は、ニューラルネットの同じ層内の全ての状態に対して、アテンションを計算します。下図に示す通り、エンコーダとデコーダがそれぞれ、セルフアテンション機構をもち、その出力が順伝搬型ニューラルネットに入力されます。このアーキテクチャは、リカレンとモデルよりも遥かに高速に学習することができ、自然言語処理の多くのブレイクスルーをもたらしました。

![任意の画像名を付ける](https://drive.google.com/uc?id=1dK1GHdJIqOvsQm1qWl3wqWq-PNC94iy5)

## Section3. 自然言語処理（NLP）における転移学習

最近のアーキテクチャにおいては、転移学習を用いるのが一般的になっています。転移学習では、ResNetのような畳み込みニューラルネットをあるタスクで学習させたあと、新しいタスクに適応させる**ファインチューニング**という技術を用います。これによって、ニューラルネットは元のタスクから学習した知識を利用することが可能になります。

アーキテクチャ的には、モデルは『ボディ』と『ヘッド』に分割し、『ヘッド』はタスクに特化したネットワークとします。学習中、『ボディ』の重みは与えられたデータの広範な特徴を学習して、これらの重みは新しいタスク用の新しいモデルを初期化するために用いられます。これによって、さまざまなタスクに対して、より少ないラベル数で、より効率的に学習できるようです。（下図イメージ）

![任意の画像名を付ける](https://drive.google.com/uc?id=1jPLnnrVRH2vgG5tgiLlxRlBAKSgjhPxA)

画像認識タスクでは、まず、[ImageNet](https://image-net.org)のような数百万の画像を含む大規模なデータセットでモデルを学習させます。このプロセスは事前学習と呼ばれ、その主な目的はエッジや色といった画像の基本的な特徴をモデルに学習させることです。

これらの事前学習済のモデルは、比較的少数のラベル付けされたサンプル（おおよそ、数百程度）を用いて、花の分類などのタスクでファインチューニングできます。このようにして学習されたモデルは、同じ量のラベル付きデータに対して、ゼロから学習した教師ありモデルよりも高い性能を示すことが可能です。

画像認識タスクでは、転移学習が標準的なアプローチとなりましたが、自然言語処理タスクではそれに類似した事前学習プロセスがなんであるかは、長年にわたって明かではありませんでした。その結果、自然言語処理タスクで高い性能を発揮するためには、一般的に大量のラベル付きデータが必要となっていました。

2017年から２０１８年において、いくつかの研究グループが新しいアプローチを提案して、ついに転移学習が自然言語処理タスクでも使えるようになりました。きっかけは、OpenAIの研究者が研究した内容で、その研究では、教師なし事前学習から抽出された特徴を使用して、感情分類タスクで高性能なモデルを達成しました。

その後、ULMFiTが登場して、さまざまなタスクに事前学習済みのLSTMモデルを適用するための、一般的なフレームワークが使用されるようになりました。

![任意の画像名を付ける](https://drive.google.com/uc?id=1_oIE5-e0Ey2_B6h1UBixGoQJu-CUCoTn)

上の図のように、ULMFiTは大きく３つのステップで構成されています。  

*   **事前学習**  
最初のステップでは、出現図m時の単語を元に次の単語を予測するタスクを学習します。このタスクを、言語モデルと言います。このアプローチは、ラベル付きデータを必要とせず、Wikipediaなどの大規模コーパスを元にしているため、比較的容易にこなすことが可能です。

*   **ドメイン適応**  
大規模コーパスで事前学習した言語モデルを、次のステップでターゲットドメインのコーパスに適応させます。例えば、上の図のように、Wikipediaで事前学習したモデルを映画レビューのIMDbコーパスを使って適応させます。この段階でも言語モデルは使われますが、今度はモデルがターゲットコーパスの次の単語を予測する必要があります。

*   **ファインチューニング**    
このステップでは、言語モデルはターゲットタスク用の分類層（映画レビューの感情の分類など）とともに、ファインチューニングされます。

ULMFiTは、自然言語処理における事前学習と転移学習のための、実行可能なフレームワークを導入することでTransformer を飛躍させるのに欠けていたものを埋めてくれました。

2018年には、セルフアテンションと転移学習を組み合わせたTransformerとして、GPT, BERTという２つのモデルがリリースされました。

## Section4. Hugging Face Transformers

新たな機械学習アーキテクチャを、新しいタスクに適用することは複雑かつ高度な作業であるため、通常は下記のようなステップでタスクを行います。



1.   PyTorch や Tensorflow をベースとした、モデルアーキテクチャを実装する
2.   サーバから事前学習済みの重みを読み込む（利用可能な場合）
3.   入力を前処理して、モデルから出力を得て、タスクに応じた後処理を実施する
4.   データローダーを実装して、モデルを学習するための損失関数とオプティマイザを定義する

🤗 Transformers では、幅広い Transformer モデルへの標準化されたインタフェースと、これらのモデルを新しいユースケースに適応するためのコードとツールを提供しています。このライブライは現在、３つの主要な深層学習フレームワーク、PyTorch, Tensorflow, JAX をサポートしており、それらの切り替えも可能となっています。

さらに、タスクに特化したヘッドを提供しているために、テキスト分類、固有表現認識、質問応答などのタスクで Transformer を簡単にファインチューニングできます。



## Section5. Transformers


すべての自然言語処理タスクは、テキストから始まります。Transformersを用いて、サンプルタスクをいくつか 実行してみたいと思います。

In [4]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

### Section5-1. テキスト分類

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 96.3 MB/s eta 0:00:00


別途、次回以降で説明しますが、Transformers には階層化されたAPIがあり、それらAPI内、**`pipeline`** から始めたいと思います。

In [3]:
#hide_output
from transformers import pipeline

classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


`text-classification` パイプラインは感情分析用に設計されたモデルを使用しますが、マルチタスクやマルチラベルの分類もサポートします

In [6]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


### Section5-2. 固有表現認識

自然言語処理では、製品、場所、人などの実世界のオブジェクトを **固有表現**といい、テキストからそれらを抽出することを、**固有表現認識(または、固有表現抽出)**といいます。

In [7]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590255,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


パイプラインがすべての固有表現を抽出し検出して、それぞれにORG(組織), LOC(場所), PER(人), などのカテゴリを割り当てています。スコアによって、その割り当ての『自信度』を表しています。

### Section5-3. 質問応答

質問応答では、**コンテキスト(context)**と呼ばれるテキストの一部と、回答を得たい質問をモデルに与えます。そして、モデルは回答に対応するテキストのスパンを返します。

In [8]:
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


パイプラインでは、単なる回答だけでなく、`start`, `end`という整数値も返しています。これらの値は、回答のスパンが見つかった文字のインデックスに対応します（固有表現認識と同じ）

質問応答にはいくつかの種類がありますが、これは別途回を設けて実施します。

今回のものは、テキストから答えを直接抽出するため、**抽出型質問応答**と呼ばれています。この方法であれば、例えば、お客様の声から関連する情報を素早く抽出して対応することができます。

一方で、長文のような苦情やクレームが山のように届き、それらをすべて読む時間がない場合は、要約モデルが役立ちます。

### Section5-4. 要約

テキスト要約の目的は、長いテキストを入力として、関連するすべての事実を含む短いテキストを生成することです。これはモデルが首尾一貫したテキストを生成する必要があるために、これまでよりもタスクは複雑となります。

In [10]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=56, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I


### Section5-5. 翻訳

翻訳も、要約と同様に生成されたテキストを出力するタスクとなります。

In [1]:
!pip install sentencepiece

※上記、コマンドを実行したら、ランタイムを再起動して下記を実行してください
![任意の画像名を付ける](https://drive.google.com/uc?id=1BHInN11F56v3SaVwA7PPvmYsAnYnZ2rF)

In [2]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-de")

translation = translator("hello, my name is Bob")

print(translation)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Hallo, mein Name ist Bob.'}]


In [5]:
translator = pipeline("translation",
                      model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Eingeschlossen sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, von Ihnen bald zu hören. Aufrichtig, Bumblebee.


### Section5-6. テキスト生成

例えば、自動保管機能を利用することで、顧客からのフィードバックに対してより素早く返答できるようにしたいとします。

In [17]:
#hide
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results

In [18]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. You have requested a new Transformers 2 action figure and the Optimus Prime action figure I received on June 17 was not a part of this purchase. After receiving your gift, I have decided to call and request a new Transformers 2 action figure, or even a new Hasbro action figure. I can confirm that in order to resolve these issues,


今回は、以上となります。